In [ ]:
from google.colab import drive
drive.mount('/content/drive/')


Mounted at /content/drive/


# ***Import and Packages***

In [ ]:
!pip install jupyter_plotly_dash
!pip install jupyter_dash
!pip install folium

     |████████████████████████████████| 512kB 4.8MB/s 
     |████████████████████████████████| 1.2MB 8.2MB/s 
     |████████████████████████████████| 51kB 6.3MB/s 
     |████████████████████████████████| 122kB 17.6MB/s 
     |████████████████████████████████| 266kB 10.5MB/s 
     |████████████████████████████████| 153kB 18.0MB/s 
     |████████████████████████████████| 81kB 7.6MB/s 
     |████████████████████████████████| 1.0MB 18.4MB/s 
     |████████████████████████████████| 3.4MB 28.5MB/s 
     |████████████████████████████████| 194kB 36.3MB/s 
     |████████████████████████████████| 1.8MB 45.8MB/s 
     |████████████████████████████████| 358kB 36.5MB/s 
  Created wheel for dpd-components: filename=dpd_components-0.1.0-cp36-none-any.whl size=4567 sha256=c8ef0691523f226d1b84dbb4867c7203bd3e2c5c702c580ce166a54d7e119005
  Stored in directory: /root/.cache/pip/wheels/36/93/4f/690abc41d258cac9a7d6246873294c1e11300dc74357d6d146
  Created wheel for idna-ssl: filename=idna_ssl-1.1.0-cp36-no

In [ ]:
from folium.plugins import MarkerCluster
import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.graph_objs as go
import pandas as pd
import folium
import plotly.figure_factory as ff
import dash_table

# ***Helper Function***

In [ ]:
def add_point_clusters(mapobj, gdf):
    # Create empty lists to contain the point coordinates and the point pop-up information
    coords, popups = [], []
    # Loop through each record in the GeoDataFrame
    for i, row in gdf.iterrows():
        # Append lat and long coordinates to "coords" list
        if row[7] == "Tampered":
          pass
        
        else:  
          coords.append([row[5], row[6]])
          # Create a string of HTML code used in the IFrame popup
          # Join together the fields in "popup_field_list" with a linebreak between them

          label ="Student_ID :"+str(row[1])+ " depth : " + str(row[2]) + " length : " + str(row[3]) + " Width : " + str(row[4])

          # Append an IFrame that uses the HTML string to the "popups" list
          popups.append(label)

    # Create a Folium feature group for this layer, since we will be displaying multiple layers
    pt_lyr = folium.FeatureGroup(name="pt_lyr")

    # Add the clustered points of crime locations and popups to this layer
    pt_lyr.add_child(MarkerCluster(locations=coords, popups=popups))

    # Add this point layer to the map object
    mapobj.add_child(pt_lyr)
    return mapobj

def clean_depth(value,remove_characters):
  for character in remove_characters:
    value = value.replace(character,".")

  return value


# ***Visualizing on Map***

In [ ]:



SF_COORDINATES = (23.8103, 90.4125)
Map = folium.Map(location=SF_COORDINATES, zoom_start=8)
Map.save("/content/drive/My Drive/Street_Picture_Data_Analysis/Dhaka_Photo_Collection.html")


app = JupyterDash(__name__)

# initial values
data_frame_pic = pd.read_csv('/content/drive/My Drive/Street_Picture_Data_Analysis/Cached_Data_pic.csv')
data_frame_untampered = data_frame_pic[data_frame_pic['Tamper Status'] != "Tampered"]
# creating options
option = list(data_frame_pic['Student_ID'].unique())
option.append('all') 
data_frame_columns = list(data_frame_pic.columns)[1:5]
data_frame_columns.append(data_frame_pic.columns[-1])
size = len(data_frame_pic)
# state management
state_value_table = data_frame_pic.to_dict('Records')
state_value_dist = ff.create_distplot([[0,1]],['depth'],show_hist=False)
state_value_dist.update_layout(title_text='Distribution of the depth')


app.layout = html.Div(
    [
        html.Div(
            [
             
                html.Label('Please Select ID',style ={'color':'black'}),
                      dcc.Dropdown(
                            id = "Data",
                            options=[
                                  {  
                                      'label': str(i), 'value': str(i)} for i in option
                                  ],
                                  value=['all'],
                                  multi=True
                                ),
                html.H1(
                    "DHAKA CITY PICTURES OF DAMAGED ROADS",
                    style={"textAlign": "center"},
                ),
                html.Iframe(
                    id="map",
                    srcDoc=open("/content/drive/My Drive/Street_Picture_Data_Analysis/Dhaka_Photo_Collection.html", "r").read(),
                    height="100%",
                    width="100%",
                ),
                html.H2(
                    id="total_collected",
                    children="Total pics collected: {}".format(size),
                    style={"textAlign": "center"},
                ),
             
                html.Div(
                  [
                                   html.Div(
                                          [
                                        dcc.Graph(
                                        id='depth',
                                        figure=state_value_dist
                                                )
                                    
                                          ],style={"width": "59%",'display': 'inline-block','float': 'left'}
                                          ),
                                  html.Div(
                                          [
                                            dash_table.DataTable(
                                            id='table',
                                            columns=[{"name": i, "id": i} for i in data_frame_columns],
                                            page_size=13,
                                            data=state_value_table,
                                            )
                                    
                                          ],style={'width': '30%', 'display': 'inline-block','float': 'right'}
                                          )

                   
                  ]
              ),
            ],
            style={"height": "70vh"},
        ),
    ],
    style={"width": "98.7%", "display": "inline-block"}
)
@app.callback(
    Output("map", "srcDoc"),
    [Input("Data", "value")],
)
def update_value(value):
  if len(value) != 0:
    if 'all' not in value:
      data_frame = data_frame_pic[data_frame_pic['Student_ID'].isin(value)]
    
      Map = folium.Map(location=SF_COORDINATES, zoom_start=8)

      Map = add_point_clusters(Map, data_frame)

      Map.save("/content/drive/My Drive/Street_Picture_Data_Analysis/Dhaka_Photo_Collection.html")
    else:
      Map = folium.Map(location=SF_COORDINATES, zoom_start=8)

      Map = add_point_clusters(Map, data_frame_pic)

      Map.save("/content/drive/My Drive/Street_Picture_Data_Analysis/Dhaka_Photo_Collection.html")

  return open("/content/drive/My Drive/Street_Picture_Data_Analysis/Dhaka_Photo_Collection.html", "r").read()


@app.callback(
    Output("table", "data"),
    Output("depth", "figure"),
    [Input("Data", "value")],
)
def count_and_grap(value):
  global state_value_dist
  global state_value_table
  if len(value) != 0:
    if 'all' not in value:
      data_frame = data_frame_untampered[data_frame_untampered['Student_ID'].isin(value)]
      # Create distplot with curve_type set to 'normal'
      figure_dist = ff.create_distplot([data_frame['Depth'].apply(lambda x : float(clean_depth(x,[","," "])) if (type(x) == type("string")) else x )],['depth'],show_hist=False)
      # Add title
      figure_dist.update_layout(title_text='Distribution of the depth')
      figure_tab = data_frame.to_dict('Records')
      state_value_dist = figure_dist
      state_value_table = figure_tab

    else:
      
      figure_dist = ff.create_distplot([data_frame_untampered['Depth'].apply(lambda x : float(clean_depth(x,[","," "])) if (type(x) == type("string")) else x )],['depth'],show_hist=False)
      figure_dist.update_layout(title_text='Distribution of the depth')
      figure_tab = data_frame_pic.to_dict('Records')
      state_value_dist = figure_dist
      state_value_table = figure_tab
    
  else:
    pass




  return state_value_table,state_value_dist



app._terminate_server_for_port("localhost", 8090)
app.run_server(mode='external',port=8090)






Dash app running on:


<IPython.core.display.Javascript object>

In [ ]:
!sudo lsof -i -P -n | grep LISTEN

node         8 root   19u  IPv6  17042      0t0  TCP *:8080 (LISTEN)
jupyter-n   25 root    3u  IPv4  17048      0t0  TCP 172.28.0.2:9000 (LISTEN)
python3   1827 root   19u  IPv4 336010      0t0  TCP 127.0.0.1:50045 (LISTEN)
python3   1827 root   23u  IPv4 336014      0t0  TCP 127.0.0.1:51407 (LISTEN)
python3   1827 root   26u  IPv4 336018      0t0  TCP 127.0.0.1:34195 (LISTEN)
python3   1827 root   29u  IPv4 336022      0t0  TCP 127.0.0.1:40437 (LISTEN)
python3   1827 root   35u  IPv4 336028      0t0  TCP 127.0.0.1:45821 (LISTEN)
python3   1827 root   48u  IPv4 334642      0t0  TCP 127.0.0.1:33355 (LISTEN)
python3   1827 root   63u  IPv4 341953      0t0  TCP 127.0.0.1:8090 (LISTEN)
